<a href="https://colab.research.google.com/github/jg4726/public-file/blob/main/object_tracking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
#making directory of yolov3, input and output
!mkdir yolov3 input output
##download pretrained weight, configs and class name to the yolov3 file
!wget https://raw.githubusercontent.com/pjreddie/darknet/master/data/coco.names -P yolov3
!wget https://raw.githubusercontent.com/pjreddie/darknet/master/cfg/yolov3.cfg -P yolov3
!wget https://pjreddie.com/media/files/yolov3.weights -P yolov3




--2022-04-18 20:36:27--  https://raw.githubusercontent.com/pjreddie/darknet/master/data/coco.names
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 625 [text/plain]
Saving to: ‘yolov3/coco.names’

coco.names          100%[===================>]     625  --.-KB/s    in 0s      

2022-04-18 20:36:27 (30.1 MB/s) - ‘yolov3/coco.names’ saved [625/625]

--2022-04-18 20:36:27--  https://raw.githubusercontent.com/pjreddie/darknet/master/cfg/yolov3.cfg
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8342 (8.1K) [text/plain]
Saving to: ‘yolov

In [ ]:
###################################################################
# Note: Google colab can't show cv2 window so click the video output below to see the video
###################################################################
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
#from google.colab.patches import cv2_imshow
from moviepy.editor import *

#Define objectness, prob and NMS thresholds
OBJ_THRESH = .5
P_THRESH = .5
NMS_THRESH = .5

# Set random seed
np.random.seed(999)

#%% Load YOLOv3 COCO weights, configs and class IDs

# Import class names
with open('yolov3/coco.names', 'rt') as f:
    classes = f.read().rstrip('\n').split('\n')
colors = np.random.randint(0, 255, (len(classes), 3))
# Give the configuration and weight files for the model and load the network using them
cfg = 'yolov3/yolov3.cfg'
weights = 'yolov3/yolov3.weights'
# Load model
model = cv2.dnn.readNetFromDarknet(cfg, weights)
# Extract names from output layers
layersNames = model.getLayerNames()
outputNames = [layersNames[i[0] - 1] for i in model.getUnconnectedOutLayers()]

#%% Define function to extract object coordinates if successful in detection
def where_is_it(frame, outputs):
    frame_h = frame.shape[0]
    frame_w = frame.shape[1]
    bboxes, probs, class_ids = [], [], []
    for preds in outputs: # different detection scales
        hits = np.any(preds[:, 5:] > P_THRESH, axis=1) & (preds[:, 4] > OBJ_THRESH)
        # Save prob and bbox coordinates if both objectness and probability pass respective thresholds 
        for i in np.where(hits)[0]:
            pred = preds[i, :]
            center_x = int(pred[0] * frame_w)
            center_y = int(pred[1] * frame_h)
            width = int(pred[2] * frame_w)
            height = int(pred[3] * frame_h)
            left = int(center_x - width / 2)
            top = int(center_y - height / 2)
            # Append all info
            temp_cls = np.argmax(pred[5:])
            #print(temp_cls)
            if temp_cls == 32:
                bboxes.append([left, top, width, height])
                probs.append(float(np.max(pred[5:])))
                class_ids.append(np.argmax(pred[5:]))
    return bboxes, probs, class_ids

vid = cv2.VideoCapture('input/ball2.avi')
vid_w, vid_h = int(vid.get(3)), int(vid.get(4))
out = cv2.VideoWriter('output/output.mp4', cv2.VideoWriter_fourcc(*'mp4v'),
                      vid.get(cv2.CAP_PROP_FPS), (vid_w, vid_h))

# Check if capture started successfully
assert vid.isOpened()

#%% Initiate processing

# Init count
count = 0

# Create new window
#cv2.namedWindow('stream')


class KalmanFilter:
    kf = cv2.KalmanFilter(4, 2)
    kf.measurementMatrix = np.array([[1, 0, 0, 0], [0, 1, 0, 0]], np.float32)
    kf.transitionMatrix = np.array([[1, 0, 1, 0], [0, 1, 0, 1], [0, 0, 1, 0], [0, 0, 0, 1]], np.float32)


    def predict(self, coordX, coordY):
        ''' This function estimates the position of the object'''
        measured = np.array([[np.float32(coordX)], [np.float32(coordY)]])
        self.kf.correct(measured)
        predicted = self.kf.predict()
        x, y = int(predicted[0]), int(predicted[1])
        return x, y

kf = KalmanFilter()

while(vid.isOpened()):
    # Perform detection every 60 frames
    perform_detection = count % 4 == 0
    ok, frame = vid.read()

    if ok:
        if perform_detection: # perform detection
            blob = cv2.dnn.blobFromImage(frame, 1 / 255, (416, 416), [0,0,0], 1, crop=False)
            # Pass blob to model
            model.setInput(blob)
            # Execute forward pass
            outputs = model.forward(outputNames)
            bboxes, probs, class_ids = where_is_it(frame, outputs)
            if len(bboxes) > 0:
            
                # Init multitracker
                mtracker = cv2.MultiTracker_create()
                # Apply non-max suppression and pass boxes to the multitracker
                idxs = cv2.dnn.NMSBoxes(bboxes, probs, P_THRESH, NMS_THRESH)
                for i in idxs:
                    bbox = [int(v) for v in bboxes[i[0]]]
                    x, y, w, h = bbox
                    # Use median flow

                    mtracker.add(cv2.TrackerMedianFlow_create(), frame, (x, y, w, h))
                # Increase counter
                count += 1
            else: # declare failure
                cv2.putText(frame, 'Detection failed', (20, 80), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0,0,255), 2)
        else: # perform tracking
            is_tracking, bboxes = mtracker.update(frame)
            if is_tracking:
                for i, bbox in enumerate(bboxes):
                    x, y, w, h = [int(val) for val in bbox]
                    class_id = classes[class_ids[idxs[i][0]]]
                    col = [int(c) for c in colors[class_ids[idxs[i][0]], :]]
                    cx = x + w / 2
                    cy = y + h / 2
                    pred_x, pred_y = kf.predict(cx, cy)
                    # Mark tracking frame with corresponding color, write class name on top
                    color =(0, 0, 255)
                    cv2.rectangle(frame, (x, y), (x+w, y+h), col, 2)
                    cv2.circle(frame, (pred_x, pred_y), 2, (255, 0, 0), 4)
                    cv2.putText(frame, 'prediction', (pred_x -10, pred_y-20),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5,(255, 0, 0), 2) 
                    cv2.circle(frame,(int(cx),int(cy)), 2, (0, 0, 255), 4)
                    cv2.putText(frame, 'centroid', (int(cx)+10,int(cy)),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5,(0, 0, 255), 2) 
                    cv2.putText(frame, class_id, (x, y - 15),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5,(0, 0, 255), 2)   
                # Increase counter
                count += 1
            # If tracking fails, reset count to trigger detection
            else:
                 
                count = 0

        # Display the resulting frame
        #cv2_imshow(frame)
        out.write(frame)
        # Press ESC to exit
        if cv2.waitKey(25) & 0xFF == 27:
            break
    # Break if capture read does not work
    else:
        print('Exhausted video capture.')
        break
out.release()
cv2.destroyAllWindows()

#output the video, change the playback speed to 0.25 for better performance
clip = VideoFileClip("/content/output/output.mp4")
   
# applying speed effect
final = clip.fx( vfx.speedx, 0.25)
  
# showing final clip
final.ipython_display()

Exhausted video capture.


100%|██████████| 165/165 [00:00<00:00, 295.24it/s]
